<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.01/colab/02_IR_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IR - Assignment 1

## Part 2 - DF IDF

In this part we will use the TF-IDF with BM25 and Okapi Algorithm

In [3]:
# get the cleaned data from the source

aj_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/A_J_word.csv?raw=true"
bbc_wrod_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/BBC_word.csv?raw=true"
jp_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/J_P_word.csv?raw=true"
nyt_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/NYT_word.csv?raw=true"

In [2]:
aj_word_file = "A_J_word.csv"
bbc_word_file = "BBC_word.csv"
jp_word_file = "J_P_word.csv"
nyt_word_file = "NYT_word.csv"

In [6]:
# download the data using requests
import requests
import pandas as pd
import os

response = requests.get(aj_word_link)
if response.status_code == 200: # if the response is 200 OK
  with open(aj_word_file, "wb") as file:
    file.write(response.content)
  print(f"Sucessfully downloaded the file into {aj_word_file}")

Sucessfully downloaded the file into A_J_word.csv


In [7]:
# open the file using pandas

df_aj = pd.read_csv(aj_word_file)
df_aj.head()

,text,title
0,pope francis has renewed calls for an immediat...,"pope renews call for gaza ceasefire, release o..."
1,united states president ronald reagan's order ...,biden is still the best us president israel co...
2,the israeli military has launched more air str...,israeli air strikes continue across gaza as tr...
3,french police officers entered the paris insti...,police remove pro-palestinian students from pa...
4,NaN,mass graves found at southern gaza hospital ra...


We will build our own TF-IDF BM25 algorithm

$$
f(q, d)=\sum_{w\in q\cap d}{\frac{(k+1)c(w,d)}{c(w,d) +k\left(1 - b +b\cdot \frac{L_d}{L_{avg}}\right)}} \cdot \log\left(\frac{M + 1}{df(w)}\right)
$$

$$b\in[0, 1], k\in[0, +\infty)$$


In [22]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [26]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [27]:
corpuse = df_aj["text"].fillna("").astype(str).tolist()

In [45]:
stop_words = set(stopwords.words("english"))

# create a global corpus with all the word in all the document
processed_corpus = []
for document in corpuse:
    tokens = word_tokenize(document)
    filtered_tokens = [w for w in tokens if not w.lower() in stop_words]
    processed_corpus.append(filtered_tokens)

# Convert processed_corpus into a list of strings
processed_corpus_str = [' '.join(doc) for doc in processed_corpus]


In [46]:
# remove all word containing number (since we don't consider them)

global_corpus = [word for word in global_corpus if not any(char.isdigit() for char in word)]

In [47]:
print(global_corpus)

['aa', 'abadi', 'abbas', 'abdallah', 'abdel', 'able', 'abroad', 'abstains', 'abu', 'abuse', 'abuses', 'abusive', 'academic', 'academy', 'accept', 'accepted', 'according', 'accusations', 'accused', 'accuses', 'accusing', 'achieving', 'acre', 'acres', 'across', 'action', 'actions', 'activated', 'activist', 'activists', 'acts', 'actually', 'adding', 'additional', 'address', 'addressed', 'aden', 'adha', 'administration', 'administrations', 'administrative', 'administrators', 'admitted', 'adopt', 'adopted', 'advice', 'adviser', 'advocates', 'advocating', 'aerial', 'affairs', 'afghanistan', 'afraid', 'africa', 'african', 'aftermath', 'agencies', 'agency', 'aggressive', 'ago', 'agree', 'agreed', 'agreement', 'aharon', 'ahead', 'aid', 'aida', 'aimed', 'aims', 'air', 'aircraft', 'airdrop', 'airports', 'airspace', 'al', 'aleppo', 'alert', 'alex', 'ali', 'aligned', 'alishba', 'alive', 'allah', 'allegations', 'alleged', 'allergic', 'alley', 'allies', 'allow', 'allowed', 'allows', 'ally', 'almost',

In [48]:
# the lenght of the total dictionary

print(len(global_corpus))

2675


In [55]:
# fill the data according to the formula

from sklearn.feature_extraction.text import TfidfVectorizer

corpuse = df_aj["text"].fillna("").astype(str).tolist()

# remove all the words containing numbers from the corpuse
corpuse = [word for word in corpuse if not any(char.isdigit() for char in word)]

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Fit the vectorizer to your processed corpus
vectorizer.fit(corpuse)

# Transform the corpus into a TF-IDF matrix
tf_idf_mat = vectorizer.transform(corpuse)

# Get the feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Convert the sparse matrix to a dense array (optional)
tf_idf_array = tf_idf_mat.toarray()

In [56]:
print(feature_names)

['aa' 'abadi' 'abbas' ... 'yusuf' 'zawayda' 'zones']


In [58]:
# save the tf_idf_mat into a csv file with on the top the feature name


import pandas as pd

# Create a pandas DataFrame from the TF-IDF matrix
df = pd.DataFrame(tf_idf_array, columns=feature_names)

# Save the DataFrame to a CSV file
df.to_csv('tf_idf_matrix.csv', index=True)